In [1]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import numpy as np
from annotations import *
from extract_video_features import *
from extract_audio_features import *
import cv2
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.metrics import f1_score
import importlib
from tqdm import tqdm
import ordinal_classification as o_c
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from util import runGridSearchClassifiers
from sklearn.model_selection import cross_val_predict

In [2]:
#get current directory
cwd = os.getcwd()

In [3]:
cwd

'/Users/valentinadiproietto/filrouge'

In [4]:
OpenFace_folder = '/Users/valentinadiproietto/OpenFace'
filename_annotations = 'https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'

In [5]:
Video_folder = '/Users/valentinadiproietto/Desktop/video_stress'

In [6]:
video_paths, video_names = get_videos(Video_folder)

In [7]:
video_names

['WIN_20210331_21_22_52_Pro',
 'WIN_20210329_14_13_45_Pro',
 'WIN_20210406_18_49_10_Pro',
 'WIN_20210408_14_11_32_Pro',
 'WIN_20210408_15_20_51_Pro',
 'WIN_20210404_10_58_27_Pro',
 'WIN_20210414_06_24_52_Pro',
 'WIN_20210406_15_06_15_Pro',
 'WIN_20210417_14_53_12_Pro',
 'WIN_20210413_15_38_01_Pro',
 'WIN_20210408_11_48_58_Pro',
 'WIN_20210408_16_04_32_Pro',
 'WIN_20210329_10_16_02_Pro',
 'WIN_20210323_19_17_40_Pro',
 'WIN_20210409_10_26_11_Pro',
 'Test_pour_AFPA',
 'WIN_20210405_15_09_16_Pro',
 'WIN_20210407_14_54_56_Pro_edit2',
 'WIN_20210406_21_05_52_Pro',
 'WIN_20210403_18_49_15_Pro',
 'WIN_20210408_14_02_19_Pro',
 'WIN_20210415_15_41_24_Pro',
 'WIN_20210406_18_35_52_Pro',
 'WIN_20210402_14_27_50_Pro',
 'WIN_20210407_09_04_05_Pro',
 'WIN_20210402_19_04_53_Pro',
 'WIN_20210416_08_06_54_Pro',
 'Video_1',
 'WIN_20210408_14_00_44_Pro',
 'WIN_20210404_21_41_12_Pro',
 'WIN_20210330_13_10_29_Pro']

In [8]:
list_dataframes = []
for i in video_names: 
    list_dataframes.append(create_dataframe_video('/Users/valentinadiproietto/OpenFace/processed/', i))


In [9]:
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210402_14_27_50_Pro')


In [10]:
len(video_names)

29

In [11]:
list_df_max = []
for v_name in tqdm(video_names):
    df_annoted = get_df_video_with_annotations('/Users/valentinadiproietto/OpenFace/processed/', v_name, filename_annotations, "max")
    list_df_max.append(eliminate_features(df_annoted))


    

100%|██████████| 29/29 [02:12<00:00,  4.58s/it]


## RECUPERO LE PREDIZIONI SULLE TIME WINDOW

In [2]:
predict5s = pd.read_csv ('au_tw5.csv')

In [3]:
predict5s = pd.read_csv('au_tw5.csv').drop("Unnamed: 0", axis =1)
predict5s

,video_name,diapo,frameTimeWindow,ypredict
0,Test_pour_AFPA,9,44,1.0
1,Test_pour_AFPA,9,45,0.0
2,Test_pour_AFPA,9,46,1.0
3,Test_pour_AFPA,9,47,1.0
4,Test_pour_AFPA,9,48,1.0
...,...,...,...,...
5136,WIN_20210417_14_53_12_Pro,18,120,1.0
5137,WIN_20210417_14_53_12_Pro,18,121,1.0
5138,WIN_20210417_14_53_12_Pro,18,122,1.0
5139,WIN_20210417_14_53_12_Pro,18,123,1.0


## PIVOT TABLE WITH COUNT OF PREDICTION 5SEC, TO PREDICT STRESS Global

In [4]:
X = predict5s.pivot_table(values='frameTimeWindow', columns='ypredict', index=['video_name'], aggfunc='count', fill_value=0)
X_sum = X.sum(axis=1).values.copy()
for col_number in range(len(X.columns)):
    X.iloc[:,col_number] = X.iloc[:,col_number] / X_sum
X

ypredict,0.0,1.0,2.0
video_name,,,
Test_pour_AFPA,0.253968,0.730159,0.015873
Video_1,0.509934,0.483444,0.006623
WIN_20210323_19_17_40_Pro,0.017857,0.964286,0.017857
WIN_20210329_10_16_02_Pro,0.061224,0.938776,0.000000
WIN_20210330_13_10_29_Pro,0.317881,0.682119,0.000000
WIN_20210331_21_22_52_Pro,0.217647,0.770588,0.011765
WIN_20210402_19_04_53_Pro,0.617143,0.371429,0.011429
WIN_20210403_18_49_15_Pro,0.171271,0.828729,0.000000
WIN_20210404_10_58_27_Pro,0.430000,0.565000,0.005000


In [5]:
#recuperalo stress globale vero 
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(['1', '8', '9', '10', '11', '12', '17', '18'], axis =1)
df_annotations_stress

,video_name,stress_global
0,Test_pour_AFPA,1.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_14_27_50_Pro,1.0
7,WIN_20210402_19_04_53_Pro,2.0
8,WIN_20210403_18_49_15_Pro,2.0
9,WIN_20210404_10_58_27_Pro,1.0


In [6]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name'])
X = Xy.iloc[:,:-1].set_index(['video_name'])
y = Xy.iloc[:,-1]
Xy

,video_name,0.0,1.0,2.0,stress_global
0,Test_pour_AFPA,0.253968,0.730159,0.015873,1.0
1,Video_1,0.509934,0.483444,0.006623,0.0
2,WIN_20210323_19_17_40_Pro,0.017857,0.964286,0.017857,1.0
3,WIN_20210329_10_16_02_Pro,0.061224,0.938776,0.000000,1.0
4,WIN_20210330_13_10_29_Pro,0.317881,0.682119,0.000000,0.0
5,WIN_20210331_21_22_52_Pro,0.217647,0.770588,0.011765,1.0
6,WIN_20210402_19_04_53_Pro,0.617143,0.371429,0.011429,2.0
7,WIN_20210403_18_49_15_Pro,0.171271,0.828729,0.000000,2.0
8,WIN_20210404_10_58_27_Pro,0.430000,0.565000,0.005000,1.0
9,WIN_20210404_21_41_12_Pro,0.560345,0.439655,0.000000,0.0


In [7]:
X

,0.0,1.0,2.0
video_name,,,
Test_pour_AFPA,0.253968,0.730159,0.015873
Video_1,0.509934,0.483444,0.006623
WIN_20210323_19_17_40_Pro,0.017857,0.964286,0.017857
WIN_20210329_10_16_02_Pro,0.061224,0.938776,0.000000
WIN_20210330_13_10_29_Pro,0.317881,0.682119,0.000000
WIN_20210331_21_22_52_Pro,0.217647,0.770588,0.011765
WIN_20210402_19_04_53_Pro,0.617143,0.371429,0.011429
WIN_20210403_18_49_15_Pro,0.171271,0.828729,0.000000
WIN_20210404_10_58_27_Pro,0.430000,0.565000,0.005000


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]



In [10]:
best_result_tw5_glob, y_predict_tw5_glob, y_proba, result_list_tw5_glob = runGridSearchClassifiers(X, y, 5, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best estimator LogisticRegression(C=1, multi_class='multinomial', random_state=42)
Best results 0.3375396825396826
Best params {'C': 1, 'class_weight': None}
accuracy (mean, std) 0.4133333333333333 0.16679994670929071
f1 (mean, std) 0.3375396825396826 0.17745570315152676
balanced accuracy (mean, std) 0.34444444444444444 0.13788526273323173
precision (mean, std) 0.33888888888888885 0.21163677245882892
recall (mean, std) 0.4133333333333333 0.16679994670929071

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=1)),
                ('logistic',
                 LogisticRegression(C=10, class_weight='balanced',
                                    multi_class='multinomial'))])
Best results 0.4003174603174603
Best params {'logistic__C': 10, 'logistic__class_weight': 'balanced', 'pca__n_components': 1}
accuracy (mean, std) 0.4333333333333333 0.2494438257849294
f1 (m

In [11]:
best_result_tw5_glob

{'best_estimator': KNeighborsClassifier(n_neighbors=10, p=1),
 'best_score': 0.44965079365079363,
 'best_params': {'n_neighbors': 10, 'p': 1, 'weights': 'uniform'},
 'mean_test_f1_score': 0.44965079365079363,
 'std_test_f1_score': 0.12500918182453,
 'mean_test_accuracy_score': 0.5199999999999999,
 'std_test_accuracy_score': 0.11274356350191841,
 'mean_test_balanced_accuracy_score': 0.4888888888888888,
 'std_test_balanced_accuracy_score': 0.08164965809277258,
 'mean_test_precision': 0.4488888888888889,
 'std_test_precision': 0.15738605519667356,
 'mean_test_recall': 0.5199999999999999,
 'std_test_recall': 0.11274356350191841}

In [12]:
y_predict_tw5_glob
##ATTENZIONE NON PREDICE MAI I 2

array([1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0.])

## ORA SEMPRE PARTENDO DALLE PEDIZIONI DI  5 SECONDI FACCIO NEL AGGIUNGENDO MINMAX sulle predizioni e predico su global

In [13]:
predict5s['ypredict'].value_counts()

1.0    3302
0.0    1670
2.0     169
Name: ypredict, dtype: int64

In [14]:
predict5s_global = predict5s.drop("diapo", axis = 1)
predict5s_global

,video_name,frameTimeWindow,ypredict
0,Test_pour_AFPA,44,1.0
1,Test_pour_AFPA,45,0.0
2,Test_pour_AFPA,46,1.0
3,Test_pour_AFPA,47,1.0
4,Test_pour_AFPA,48,1.0
...,...,...,...
5136,WIN_20210417_14_53_12_Pro,120,1.0
5137,WIN_20210417_14_53_12_Pro,121,1.0
5138,WIN_20210417_14_53_12_Pro,122,1.0
5139,WIN_20210417_14_53_12_Pro,123,1.0


In [15]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [16]:

X_glob_minmax = predict5s_global.groupby(['video_name']).agg({'ypredict':['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})

In [17]:
X_glob_minmax

ypredict                             \
                                     mean  min  max median       std   
video_name                                                             
Test_pour_AFPA                   0.761905  0.0  2.0    1.0  0.462910   
Video_1                          0.496689  0.0  2.0    0.0  0.514771   
WIN_20210323_19_17_40_Pro        1.000000  0.0  2.0    1.0  0.189547   
WIN_20210329_10_16_02_Pro        0.938776  0.0  1.0    1.0  0.240561   
WIN_20210330_13_10_29_Pro        0.682119  0.0  1.0    1.0  0.467202   
WIN_20210331_21_22_52_Pro        0.794118  0.0  2.0    1.0  0.433741   
WIN_20210402_19_04_53_Pro        0.394286  0.0  2.0    0.0  0.513016   
WIN_20210403_18_49_15_Pro        0.828729  0.0  1.0    1.0  0.377790   
WIN_20210404_10_58_27_Pro        0.575000  0.0  2.0    1.0  0.505622   
WIN_20210404_21_41_12_Pro        0.439655  0.0  1.0    0.0  0.497418   
WIN_20210405_15_09_16_Pro        0.754839  0.0  1.0    1.0  0.431577   
WIN_20210406_15_06_15_Pro        0.678049  0.0  1.0    1.0  0.468368   
WIN_20210406_18_35_52_Pro        0.829114  0.0  1.0    1.0  0.377606   
WIN_20210406_18_49_10_Pro        0.771930  0.0  2.0    1.0  0.460851   
WIN_20210406_21_05_52_Pro        0.994595  0.0  2.0    1.0  0.221097   
WIN_20210407_09_04_05_Pro        0.052356  0.0  2.0    0.0  0.285403   
WIN_20210407_14_54_56_Pro_edit2  0.584906  0.0  1.0    1.0  0.494295   
WIN_20210408_11_48_58_Pro        0.720000  0.0  2.0    1.0  0.469422   
WIN_20210408_14_00_44_Pro        0.488764  0.0  1.0    0.0  0.501284   
WIN_20210408_14_02_19_Pro        1.000000  1.0  1.0    1.0  0.000000   
WIN_20210408_14_11_32_Pro        0.480769  0.0  1.0    0.0  0.501239   
WIN_20210408_15_20_51_Pro        1.382353  0.0  2.0    2.0  0.769417   
WIN_20210408_16_04_32_Pro        0.729412  0.0  2.0    1.0  0.519176   
WIN_20210409_10_26_11_Pro        0.913295  0.0  1.0    1.0  0.282219   
WIN_20210413_15_38_01_Pro        0.815029  0.0  1.0    1.0  0.389401   
WIN_20210414_06_24_52_Pro        0.787565  0.0  1.0    1.0  0.410095   
WIN_20210415_15_41_24_Pro        0.442396  0.0  2.0    0.0  0.507034   
WIN_20210416_08_06_54_Pro        0.895706  0.0  2.0    1.0  0.614772   
WIN_20210417_14_53_12_Pro        0.320611  0.0  2.0    0.0  0.484644   

                                                                              
                                percentil25 percentil75   kurtosis      skew  
video_name                                                                    
Test_pour_AFPA                          0.0         1.0  -0.243775 -0.735162  
Video_1                                 0.0         1.0  -1.616264  0.160339  
WIN_20210323_19_17_40_Pro               1.0         1.0  25.000000  0.000000  
WIN_20210329_10_16_02_Pro               1.0         1.0  11.398551 -3.660403  
WIN_20210330_13_10_29_Pro               0.0         1.0  -1.388147 -0.782210  
WIN_20210331_21_22_52_Pro               1.0         1.0   0.225356 -1.009390  
WIN_20210402_19_04_53_Pro               0.0         1.0  -0.942580  0.687488  
WIN_20210403_18_49_15_Pro               1.0         1.0   1.045376 -1.745101  
WIN_20210404_10_58_27_Pro               0.0         1.0  -1.670073 -0.186337  
WIN_20210404_21_41_12_Pro               0.0         1.0  -1.940875  0.243157  
WIN_20210405_15_09_16_Pro               1.0         1.0  -0.596266 -1.184793  
WIN_20210406_15_06_15_Pro               0.0         1.0  -1.419119 -0.762155  
WIN_20210406_18_35_52_Pro               1.0         1.0   1.057959 -1.748702  
WIN_20210406_18_49_10_Pro               1.0         1.0  -0.103975 -0.739449  
WIN_20210406_21_05_52_Pro               1.0         1.0  17.534434 -0.430654  
WIN_20210407_09_04_05_Pro               0.0         0.0  33.996628  5.797763  
WIN_20210407_14_54_56_Pro_edit2         0.0         1.0  -1.881232 -0.344628  
WIN_20210408_11_48_58_Pro               0.0         1.0  -0.801367 -0.717951  
WIN_20210408_14_00_44_Pro               0.0         1.0  -1.997979  0

In [18]:
#questo e' lo stress vero per diapositiva
#recuperalo stress globale vero 
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(['1', '8', '9', '10', '11', '12', '17', '18'], axis =1)
df_annotations_stress

,video_name,stress_global
0,Test_pour_AFPA,1.0
1,Video_1,0.0
2,WIN_20210323_19_17_40_Pro,1.0
3,WIN_20210329_10_16_02_Pro,1.0
4,WIN_20210330_13_10_29_Pro,0.0
5,WIN_20210331_21_22_52_Pro,1.0
6,WIN_20210402_14_27_50_Pro,1.0
7,WIN_20210402_19_04_53_Pro,2.0
8,WIN_20210403_18_49_15_Pro,2.0
9,WIN_20210404_10_58_27_Pro,1.0


In [19]:
X_glob_minmaxy = X_glob_minmax.merge(df_annotations_stress, how='inner', on=['video_name'])
X_glob_minmax = X_glob_minmaxy.iloc[:,:-1].set_index(['video_name'])
y_glob_minmax = X_glob_minmaxy.iloc[:,-1]
X_glob_minmaxy

,video_name,"(ypredict, mean)","(ypredict, min)","(ypredict, max)","(ypredict, median)","(ypredict, std)","(ypredict, percentil25)","(ypredict, percentil75)","(ypredict, kurtosis)","(ypredict, skew)",stress_global
0,Test_pour_AFPA,0.761905,0.0,2.0,1.0,0.462910,0.0,1.0,-0.243775,-0.735162,1.0
1,Video_1,0.496689,0.0,2.0,0.0,0.514771,0.0,1.0,-1.616264,0.160339,0.0
2,WIN_20210323_19_17_40_Pro,1.000000,0.0,2.0,1.0,0.189547,1.0,1.0,25.000000,0.000000,1.0
3,WIN_20210329_10_16_02_Pro,0.938776,0.0,1.0,1.0,0.240561,1.0,1.0,11.398551,-3.660403,1.0
4,WIN_20210330_13_10_29_Pro,0.682119,0.0,1.0,1.0,0.467202,0.0,1.0,-1.388147,-0.782210,0.0
5,WIN_20210331_21_22_52_Pro,0.794118,0.0,2.0,1.0,0.433741,1.0,1.0,0.225356,-1.009390,1.0
6,WIN_20210402_19_04_53_Pro,0.394286,0.0,2.0,0.0,0.513016,0.0,1.0,-0.942580,0.687488,2.0
7,WIN_20210403_18_49_15_Pro,0.828729,0.0,1.0,1.0,0.377790,1.0,1.0,1.045376,-1.745101,2.0
8,WIN_20210404_10_58_27_Pro,0.575000,0.0,2.0,1.0,0.505622,0.0,1.0,-1.670073,-0.186337,1.0
9,WIN_20210404_21_41_12_Pro,0.439655,0.0,1.0,0.0,0.497418,0.0,1.0,-1.940875,0.243157,0.0


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]



In [20]:
best_result_tw5_glob_minmax, y_predict_tw5_glob_minmax, y_proba, result_list_tw5_glob_minmax = runGridSearchClassifiers(X_glob_minmax, y_glob_minmax, 5, models_list, parameters_list,output_predict=True, n_jobs=-1, verbose=True)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.3038888888888889
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4133333333333333 0.07483314773547883
f1 (mean, std) 0.3038888888888889 0.09371681826491533
balanced accuracy (mean, std) 0.3444444444444444 0.0888888888888889
precision (mean, std) 0.26888888888888884 0.12360830209043003
recall (mean, std) 0.4133333333333333 0.07483314773547883

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best estimator Pipeline(steps=[('pca', PCA(n_components=3)),
                ('logistic',
                 LogisticRegression(C=1, class_weight='balanced',
                                    multi_class='multinomial'))])
Best results 0.3394444444444444
Best params {'logistic__C': 1, 'logistic__class_weight': 'balanced', 'pca__n_components': 3}
accuracy (mean, std) 0.4133333333333333 0.22370615647416692
f1

In [21]:
best_result_tw5_glob_minmax

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=2)),
                 ('knn', KNeighborsClassifier(n_neighbors=12))]),
 'best_score': 0.5372698412698413,
 'best_params': {'knn__n_neighbors': 12,
  'knn__p': 2,
  'knn__weights': 'uniform',
  'pca__n_components': 2},
 'mean_test_f1_score': 0.5372698412698413,
 'std_test_f1_score': 0.13673404677140677,
 'mean_test_accuracy_score': 0.62,
 'std_test_accuracy_score': 0.1240071682515885,
 'mean_test_balanced_accuracy_score': 0.5777777777777777,
 'std_test_balanced_accuracy_score': 0.07535922203472518,
 'mean_test_precision': 0.5083333333333334,
 'std_test_precision': 0.1589898669028243,
 'mean_test_recall': 0.62,
 'std_test_recall': 0.1240071682515885}

In [22]:
y_predict_tw5_glob_minmax

array([1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.])